# Python для анализа данных

## Работа с форматами XML и JSON


# XML

До этого мы с вами собирали данные вручную, обращаясь к html страницам, размеченным для отображения в браузере. Но данные также можно собирать и через API -  — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером. 

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). Например, если бы мы хотели описать в виде XML некоторую студенческую группу, это могло бы выглядеть так:

```xml
<group>
    <number>134</number>
    <student>
        <firstname>Виталий</firstname>
        <lastname>Иванов</lastname>
    </student>
    <student>
        <firstname>Мария</firstname>
        <lastname>Петрова</lastname>
    </student>
</group>
```

Для обработки XML-файлов можно использовать тот же пакет *Beautiful Soup*, который мы уже использовали для работы с HTML. Единственное различие — нужно указать дополнительный параметр `feautres="xml"` при вызове функции `BeautifulSoup` — чтобы он не искал в документе HTML-теги.

In [1]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [2]:
!pip install lxml

In [4]:
from bs4 import BeautifulSoup

obj = BeautifulSoup(group, features="lxml")
print(obj.prettify())

<html>
 <body>
  <group>
   <number>
    134
   </number>
   <student>
    <firstname>
     Виталий
    </firstname>
    <lastname>
     Иванов
    </lastname>
   </student>
   <student>
    <firstname>
     Мария
    </firstname>
    <lastname>
     Петрова
    </lastname>
   </student>
  </group>
 </body>
</html>


Номер группы можно найти, например, вот так - для каждого объекта через точку указываем его атрибут, в который надо спуститься. 

In [5]:
obj.group.number.text # последний атрибут текст, точно также как делали в html

'134'

Но это работает только тогда, когда тэг уникальный. В других случаях, парсер всегда будет попадать в первый child-тэг, который он встретил по пути вниз.

In [6]:
obj.group.student.lastname.text # до Петровой так не добраться

'Иванов'

Перечислить всех студентов можно с помощью цикла (похожая структура у нас была и в обработке html).

In [7]:
for student in obj.group.find_all('student'):
    print(student.lastname.text, student.firstname.text)

Иванов Виталий
Петрова Мария


По сути, главное отличие xml от html, что работать вы будете не со стандартизированными структурами. Поэтому перед работой придется поиграть в детективов - запросить данные и внимательно изучить расположение узлов, чтобы понять, какие тэги вас интересуют.

XML легко представить в виде дерева, где есть главный узел (parent) и его "дети".

![](https://www.py4e.com/images/xml-tree.svg)  
*Источник: Python for Everybody, C.Severance*

Кроме BS парсить xml можно и с помощью других библиотек. Например, ElementTree.


In [8]:
import xml.etree.ElementTree as ET

In [9]:
tree = ET.fromstring(group)
list(tree) # посмотрим, что внутри

[<Element 'number' at 0x7fa808031a40>,
 <Element 'student' at 0x7fa8080319f0>,
 <Element 'student' at 0x7fa808031900>]

Cинтаксис очень похож на BS. Добрались до первой фамилии.

In [10]:
tree.find('student').find('lastname').text

'Иванов'

In [11]:
for element in tree.findall('student'):
    print(element)
    print(element.find('lastname').text)

<Element 'student' at 0x7fa8080319f0>
Иванов
<Element 'student' at 0x7fa808031900>
Петрова


Можно немного упростить код, включив дочерний тэг в findall.

In [11]:
for element in tree.findall('student/lastname'):
    print(element.text)

Иванов
Петрова


# Задача
По ссылке данные в формате xml.
http://py4e-data.dr-chuck.net/comments_42.xml

Посчитайте все комментарии в этом документе (поля count).

In [13]:
import requests
data = requests.get('http://py4e-data.dr-chuck.net/comments_42.xml').text

tree = ET.fromstring(data)

total = 0
for element in tree.findall('comments/comment/count'):
#     print(type(element.text))
    total += int(element.text)
    
print(total)

2553


In [14]:
sum([int(element.text) for element in tree.findall('comments/comment/count')])

2553

# JSON

Другой популярный формат, в котором клиент может отдать вам данные - json. JSON расшифровывается как JavaScript Object Notation и изначально возник как подмножество языка JavaScript (пусть вас не вводит в заблуждение название, этот язык ничего не имеет общего с Java), используемое для описания объектов, но впоследствии стал использоваться и в других языках программирования, включая Python. Различные API могут поддерживать либо XML, либо JSON, либо и то, и другое, так что нам полезно научиться работать с обоими типами данных (например, wiki api могла бы выгрузить нам данные и в формате json при соответствующем запросе).

JSON очень похож на описание объекта в Python и смысл квадратных и фигурных скобок такой же. Правда, есть и отличия: например, в Python одинарные и двойные кавычки ничем не отличаются, а в JSON можно использовать только двойные. Мы видим, что полученный нами JSON представляет собой словарь, значения которого — строки или числа, а также списки или словари, значения которых в свою очередь также могут быть строками, числами, списками, словарями и т.д. То есть получается такая довольно сложная структура данных.

В данный момент тот факт, что перед нами сложная структура данных, видим только мы — с точки зрения Python, j.text это просто такая строка. Однако в модуле requests есть метод, позволяющий сразу выдать питоновский объект (словарь или список), если результат запроса возвращён в формате JSON. Так что нам не придётся использовать никакие дополнительные библиотеки.

Преимущества JSON в том, что мы получаем готовый объект Python и нет необходимости использовать какие-то дополнительные библиотеки для того, чтобы с ним работать. Недостатком является то же самое: зачастую поиск информации в XML-файле может проводиться более эффективно, чем в JSON. 

## JSON (парсинг VK)

Как уже говорилось выше, не все API открытые. Так, чтобы достать информацию из vk вам придется сгенерировать токен с помощью вашего аккаунта. API VK отдает данные в json. Структура тут будет посложнее, чем то, что мы уже видели, поэтому давайте еще потренируемся.

Теперь научимся еще и загружать JSON файл с диска. Для этого нам понадобится модуль json.

In [15]:
import json
with open('vk.json',  'r', encoding='Utf-8') as json_data:
    res_loaded = json.load(json_data) # считываем данные с помощью функции .load()

Здесь у нас выгрузка постов со стены группы ВШЭ.

In [16]:
res_loaded

{'count': 1542,
 'items': [{'id': 32494,
   'from_id': -132,
   'owner_id': -132,
   'date': 1541422799,
   'marked_as_ads': 0,
   'post_type': 'post',
   'text': '',
   'attachments': [{'type': 'link',
     'link': {'url': 'http://family.hse.ru/event/view/2271',
      'title': 'День карьеры факультета бизнеса и менеджмента',
      'caption': 'family.hse.ru',
      'description': '',
      'photo': {'id': 456239093,
       'album_id': -2,
       'owner_id': 100,
       'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
       'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
       'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
       'width': 150,
       'height': 80,
       'text': '',
       'date': 1541422799}}}],
   'post_source': {'type': 'vk'},
   'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
   'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
  

In [17]:
res_loaded['items'][:2]

[{'id': 32494,
  'from_id': -132,
  'owner_id': -132,
  'date': 1541422799,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': '',
  'attachments': [{'type': 'link',
    'link': {'url': 'http://family.hse.ru/event/view/2271',
     'title': 'День карьеры факультета бизнеса и менеджмента',
     'caption': 'family.hse.ru',
     'description': '',
     'photo': {'id': 456239093,
      'album_id': -2,
      'owner_id': 100,
      'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
      'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
      'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
      'width': 150,
      'height': 80,
      'text': '',
      'date': 1541422799}}}],
  'post_source': {'type': 'vk'},
  'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
  'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
  'reposts': {'count': 2, 'user_reposted': 0},
  'vi

In [18]:
res_loaded.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [22]:
res_loaded['items'][50] # первый элемент items - первый пост со всей информацией о нем

{'id': 32099,
 'from_id': -132,
 'owner_id': -132,
 'date': 1531760787,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'copy_history': [{'id': 1050401,
   'owner_id': -79138567,
   'from_id': -79138567,
   'date': 1531760410,
   'post_type': 'post',
   'text': 'В России есть проблемы с образованием, но часть из них можно решить с помощью грамотного менеджмента.\n\nМагистерская программа НИУ ВШЭ «Экономика и управление образованием» сделает из вас крутого специалиста. Вы научитесь руководить большими образовательными организациями и открывать собственные: школы, курсы, университеты и так далее. \n\nЗадавайте в Прямой линии свои вопросы о поступлении, будущей профессии и образовательном менеджменте:\n\nhttps://thequestion.ru/interview/5663/magistratura-niu-vshe-ekonomika-i-upravlenie-obrazovaniem#',
   'attachments': [{'type': 'doc',
     'doc': {'id': 469433958,
      'owner_id': 27546726,
      'title': 'school3.gif',
      'size': 378724,
      'ext': 'gif',
      'url': 'ht

Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста и дате публикации. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [23]:
items = res_loaded['items']
full_list = []

for item in items:
    l = [item['from_id'], item['id'], item['text'], item['date']] # нужные поля
    full_list.append(l) # добавляем в список списков full_list
# несколько элементов списка
full_list[0:4]

[[-132, 32494, '', 1541422799],
 [-132,
  32493,
  'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
  1541076555],
 [-132, 32492, '', 1540743239],
 [-132, 32491, '', 1540551399]]

Видно, что в двух первых постах текста не обнаружено, там только картинки, ссылки и репосты.

Из этого списка списков можно легко сделать датафрейм `pandas`.

Но если у ссылок, репостов и картинок нет текста, то наш DataFrame будет выглядет неполноценно. Поэтому добавим условие, что если текст отсутсвует, вставим в DataFrame строку с этой отметкой.

In [24]:
# опять выберем только нужные поля
full_list = []
for item in items:
    if item['text'] == "":
        l = [item['from_id'], item['id'], "Картинка, ссылка или репост", item['date']]
    else:
        l = [item['from_id'], item['id'], item['text'], item['date']]
    full_list.append(l)

Оставлось превратить обновленный список `items` (список списков) в датафрейм. Импортируем `pandas`.

In [25]:
import pandas as pd

Создадим датафрейм:

In [26]:
df = pd.DataFrame(full_list)
df.head(10)

0      1                                                  2           3
0 -132  32494                        Картинка, ссылка или репост  1541422799
1 -132  32493  На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...  1541076555
2 -132  32492                        Картинка, ссылка или репост  1540743239
3 -132  32491                        Картинка, ссылка или репост  1540551399
4 -132  32488                        Картинка, ссылка или репост  1540302306
5 -132  32486  26-28 октября в Москве пройдет крупнейший в Ро...  1539863523
6 -132  32485                        Картинка, ссылка или репост  1539781575
7 -132  32484                        Картинка, ссылка или репост  1539718883
8 -132  32481                        Картинка, ссылка или репост  1539088885
9 -132  32480  Будущим магистрам! Новая программа Факультета ...  1539088745

Ура! Осталось только дать внятные названия столбцам и разобраться, почему дата представлена в таком виде. что делать со столбцами, мы уже знаем.

In [27]:
df.columns = ['From_id', 'Id', 'Text', 'Date_Unix']
df.head(10)

From_id     Id                                               Text  \
0     -132  32494                        Картинка, ссылка или репост   
1     -132  32493  На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...   
2     -132  32492                        Картинка, ссылка или репост   
3     -132  32491                        Картинка, ссылка или репост   
4     -132  32488                        Картинка, ссылка или репост   
5     -132  32486  26-28 октября в Москве пройдет крупнейший в Ро...   
6     -132  32485                        Картинка, ссылка или репост   
7     -132  32484                        Картинка, ссылка или репост   
8     -132  32481                        Картинка, ссылка или репост   
9     -132  32480  Будущим магистрам! Новая программа Факультета ...   

    Date_Unix  
0  1541422799  
1  1541076555  
2  1540743239  
3  1540551399  
4  1540302306  
5  1539863523  
6  1539781575  
7  1539718883  
8  1539088885  
9  1539088745

С датой все интереснее. То, что указано в столбце `date`, это дата в виде UNIX-времени (POSIX-времени). Это число секунд, прошедших с 1 января 1970 года. Несмотря на то, что такой формат даты-времени кажется необычным, он довольно широко распространен в разных системах и приложениях. Этот факт, конечно, радует, но хочется получить дату в более человеческом формате. Давайте напишем функцию для перевода UNIX-времени в формат год-месяц-день-часы-минуты-секунды. Для этого нам понадобится модуль datetime.

In [28]:
from datetime import datetime

In [29]:
def date_norm(date):
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:минуты:секунды
    date_norm, time_norm = str_d.split(' ') # разобьем результат на части, отделим дату от времени
    return date_norm, time_norm

Применим нашу функцию к элементам столбца date и создадим новый ‒ `date_norm`.

In [30]:
df['Date_Norm'] = df.Date_Unix.apply(date_norm)

In [31]:
df.head()

From_id     Id                                               Text  \
0     -132  32494                        Картинка, ссылка или репост   
1     -132  32493  На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...   
2     -132  32492                        Картинка, ссылка или репост   
3     -132  32491                        Картинка, ссылка или репост   
4     -132  32488                        Картинка, ссылка или репост   

    Date_Unix               Date_Norm  
0  1541422799  (2018-11-05, 15:59:59)  
1  1541076555  (2018-11-01, 15:49:15)  
2  1540743239  (2018-10-28, 19:13:59)  
3  1540551399  (2018-10-26, 13:56:39)  
4  1540302306  (2018-10-23, 16:45:06)

Можно было, конечно, не разбивать на части дату и время, сохранять одной строкой. А можно написать функции, которые будут отделять дату от времени ‒ извлекать их из кортежа в date_norm.

In [32]:
def get_date(date):
    return date[0]

def get_time(date):
    return date[1]

In [33]:
df['Date'] = df.Date_Norm.apply(get_date)
df['Time'] = df.Date_Norm.apply(get_time)
df.head()

From_id     Id                                               Text  \
0     -132  32494                        Картинка, ссылка или репост   
1     -132  32493  На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...   
2     -132  32492                        Картинка, ссылка или репост   
3     -132  32491                        Картинка, ссылка или репост   
4     -132  32488                        Картинка, ссылка или репост   

    Date_Unix               Date_Norm        Date      Time  
0  1541422799  (2018-11-05, 15:59:59)  2018-11-05  15:59:59  
1  1541076555  (2018-11-01, 15:49:15)  2018-11-01  15:49:15  
2  1540743239  (2018-10-28, 19:13:59)  2018-10-28  19:13:59  
3  1540551399  (2018-10-26, 13:56:39)  2018-10-26  13:56:39  
4  1540302306  (2018-10-23, 16:45:06)  2018-10-23  16:45:06

Всё! Материалы о разных методах и функциях для `vk.api` можно найти в [официальной документации](https://vk.com/dev/manuals).

## API YouTube

+ [Инструкция](https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb) по получению ключа API YouTube
+ Официальная [документация](https://developers.google.com/youtube/v3/docs) к API YouTube
+ Статья на Хабр. [Что такое API?](https://habr.com/ru/post/464261/)
+ Статья на Хабр. [Что такое JSON](https://habr.com/ru/post/554274/)

API (Application programming interface) — это контракт, который предоставляет программа. Иными словами, это функции от разработчика, которые предоставлены в наше использование и распоряжение.

Сегодня попробуем потыкаться в API YouTube и вытащить какую-нибудь информацию. Но для этого нам нужен собственно доступ к API (ключ). Как его получить, можно узнать в [инструкции](https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb).

In [2]:
!pip install google-api-python-client
import googleapiclient.discovery as api

     |████████████████████████████████| 11.2 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 88.0 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 42.4 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 397 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 223 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 36.5 MB/s  eta 0:00:01


In [26]:
# Поскольку API-ключи привязаны к личным аккаунтам и хранят эту связь, не стоит выкладывать в открытый доступ ваш ключ.
# Я создала отдельный файл youtube_key.py, где задала константу YT_API_KEY и положила свой токен в виде строки.
# Получить свой токен: https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb

# from youtube_key import *
# key = YT_API_KEY

In [24]:
key = 'ВАШ КЛЮЧ'
youtube = api.build('youtube', 'v3', developerKey = key)

Далее используем запрос, который предлагается нам в самой документации. Этот тот момент, где вам не нужно задумываться о том, почему метод называется так и почему он хочет определенные параметры — просто так придумал разработчик.

### Видео по опеределенному запросу

Например, в самой [документации](https://developers.google.com/youtube/v3/docs/search/list?hl=en) про параметр `part` сказано Set the parameter value to snippet.

`maxResults` — сколько видео (максимально можно 50, но, спойлер, это ограничение можно обойти) \
`q` — запрос, который мы ищем \
`type` — пока мы ищем видео, но можно и каналы \
`order` — по релевантности (его можно убрать) \
Это все наш запрос с помощью метода .`search()`. Выгружаем его командой `.execute()`.

In [25]:
request_search = youtube.search().list(
    part = "snippet",
    maxResults = 10,
    q = "рестораны Москвы",
    type = 'video',
    order = 'relevance')

response_search = request_search.execute()
response_search

{'kind': 'youtube#searchListResponse',
 'etag': 'nQX5Z_LgPY9CiPHIZsw16f_iugU',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'RU',
 'pageInfo': {'totalResults': 830899, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'fFIB7udAlqO7K0xFVx94ZnCjYZU',
   'id': {'kind': 'youtube#video', 'videoId': 'DuNsxZdRqs4'},
   'snippet': {'publishedAt': '2023-03-30T10:08:31Z',
    'channelId': 'UCVq_U7ZYmaTNK5VPu0_VyBw',
    'title': 'Рестораны Москвы | Трейлер | PREMIER',
    'description': 'Вместе с самыми известными рестораторами и звёздами гида «Michelin» Сергей Минаев отправляется в большое ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/DuNsxZdRqs4/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/DuNsxZdRqs4/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/DuNsxZdRqs4/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'channelTitle'

Это просто словарь! А с ними вы работать уже умеете.

К слову, использование этого API **бесплатно**, но не бесконечно. Есть определенный лимит для одного ключа —10.000 у.е в день (например, запрос видео выше стоил нам 100 у.е). Подробнее об этом можно почитать в самой документации. Такое придумано в API YouTube (для других API может быть что-то свое или вообще не быть лимита).

Посмотрим, какие тут есть ключи:

In [5]:
response_search.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'regionCode', 'pageInfo', 'items'])

Из них нас интересуют только `nextPageToken`, `pageInfo`, `items`.

`nextPageToken` — адрес следующей страницы нашего поиска; до тех пор пока он есть в выдаче, мы можем собирать данные. \
`pageInfo` — сколько всего результатов и сколько всего результатов по нашему запросу. Если всего результатов очень много, то все вы не получите (так уж устроен API YouTube); можно исхитриться и в цикле делать запрос от и до определенной даты. \
`items` — все видео, которые мы нашли (у нас их десять).

In [6]:
print(type(response_search['items'])) # это список! 
print(len(response_search['items']))  # наши десять видео

<class 'list'>
10


Вытащим самое первое видео и посмотрим, что там.

In [7]:
response_search['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'fFIB7udAlqO7K0xFVx94ZnCjYZU',
 'id': {'kind': 'youtube#video', 'videoId': 'DuNsxZdRqs4'},
 'snippet': {'publishedAt': '2023-03-30T10:08:31Z',
  'channelId': 'UCVq_U7ZYmaTNK5VPu0_VyBw',
  'title': 'Рестораны Москвы | Трейлер | PREMIER',
  'description': 'Вместе с самыми известными рестораторами и звёздами гида «Michelin» Сергей Минаев отправляется в большое ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/DuNsxZdRqs4/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/DuNsxZdRqs4/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/DuNsxZdRqs4/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Онлайн-кинотеатр PREMIER',
  'liveBroadcastContent': 'none',
  'publishTime': '2023-03-30T10:08:31Z'}}

In [8]:
response_search['items'][0].keys()

dict_keys(['kind', 'etag', 'id', 'snippet'])

Тут нас интересуют ключи `id` и `snippet`.

* в `id` записано `'videoId'` (оно нам нужно, чтобы вытащить информацию о конкретном видео, например, статистику по лайкам или комментарии)
* `snippet` содержит основную информацию о видео (id канала, заголовок, описание).

In [9]:
response_search['items'][0]['id']['videoId']

'DuNsxZdRqs4'

К слову, само видео можно посмотреть так: https://www.youtube.com/watch?v=DuNsxZdRqs4. Обратите внимание, куда именно подставлятся наше id.

## Комментарии к определенному видео

Например, можно забрать комментарии определенного видео (а в перспективе — посмотреть их тональность). Поможет нам в этом метод `.commentThreads()`.

In [11]:
comment_request = youtube.commentThreads().list(
    part = 'snippet',
    videoId = "DskGDinJbOE"
)

comment_response = comment_request.execute()
comment_response

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'U-sjGDOM0zUAqzoZ1tu0JHi1k2c',
 'nextPageToken': 'QURTSl9pM084dEFrX2JfckJQMGNTYmxCMFoyYTJvSm1saTBaYVZURWIzSHFQLVFXaTNUQVduaGFDWkhJTjJHZWU5anZJR0N2VkdMMWlnaWlueXRpd2E4Z0FxZFNYZ2Rm',
 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20},
 'items': [{'kind': 'youtube#commentThread',
   'etag': 'eEZREs6oKDV2uhTB3vXfwVi2LY0',
   'id': 'UgzbZ7YhY4NLOFIbJ6F4AaABAg',
   'snippet': {'videoId': 'DskGDinJbOE',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'W7ilr2QMxjqDN-M3ERLc5aTtSwY',
     'id': 'UgzbZ7YhY4NLOFIbJ6F4AaABAg',
     'snippet': {'videoId': 'DskGDinJbOE',
      'textDisplay': 'Гид Мишлен Пришел в Россию, чтобы Вручить Звезды Лучшим Московским Ресторанам:\r<br><a href="https://www.youtube.com/watch?v=JaYZ6-7LI4A">https://www.youtube.com/watch?v=JaYZ6-7LI4A</a>',
      'textOriginal': 'Гид Мишлен Пришел в Россию, чтобы Вручить Звезды Лучшим Московским Ресторанам:\r\nhttps://www.youtube.com/watch?v=JaYZ6-7LI4A',


In [12]:
for comment in comment_response['items']:
    print(comment['snippet']['topLevelComment']['snippet']['textOriginal'])

Гид Мишлен Пришел в Россию, чтобы Вручить Звезды Лучшим Московским Ресторанам:
https://www.youtube.com/watch?v=JaYZ6-7LI4A
Белий Кролик ти сегодня взорвешся
СДЕЛАЙ ПОЖАЛУСТА ПРО ТБИЛИСИ 🇬🇪 НО НЕ ДОРОГИЕ РЕСТОРАНЫ,А ВКУСНЫЕ!!!
Благодарю за ответ и желаю Всем Добра
Про Казань тоже напишите 🙏🤗
Была в кафе Пушкин, приятно удивлена, еда вкусная, порции большие, обслуживание на высшем уровне...🥂👍
Средний чек, про который вы говорите, нужно умножить на 2 как минимум
Ресторан Sixty не хватает
Надо показать кухня где мойка там интереснее
Азиаты готовят даже лучше чем европейцы или русские
А можно список ресторанов, которые лучшие в плане поесть,  а не в плане кинуть понты и запостить фото в запрещённую соцсеть?
Не знаю как на вкус, но интерьеры практически у всех ресторанов ( за малым исключением) - убогие и неприятные
А  ресторан "Шор Хаус" на Крокус Сити? Мой шеф там работал. 🤗
В Москве нужно привлекать советской кухней,а не иностранной.
Прекрасная кругом русская кухня,щи да лапти...а варят в

## Статистика по определенному видео
Последний метод в этом погружении в API YouTube — `.videos()`

In [14]:
video_request = youtube.videos().list(
    part = ['snippet', 'contentDetails', 'id', 'statistics'],
    id = 'DskGDinJbOE'
)

video_response = video_request.execute()
video_response

{'kind': 'youtube#videoListResponse',
 'etag': 'YRz9m_LunV763cpgfMegy_BZMwk',
 'items': [{'kind': 'youtube#video',
   'etag': 'jaUjKr5x0jKB0Sv03VDL2uBpzFY',
   'id': 'DskGDinJbOE',
   'snippet': {'publishedAt': '2021-02-27T11:58:57Z',
    'channelId': 'UCQM0TC4icu74AJ3h7N4hmTg',
    'title': 'Топ 10 | Лучшие Рестораны Москвы | Обзор Ресторанов Москвы',
    'description': 'Ресторан – это не только заведение, где можно вкусно пообедать. Место для свиданий и деловых встреч, место, где можно приятно провести время, а некоторые рестораны становятся настоящими достопримечательностями. Они предлагают не просто еду, а новый опыт, вкусовые ощущения и особенную атмосферу.\n\nЗдесь дорогая обстановка, уникальные, зачастую экзотические блюда из лучших продуктов, а повара известны своим искусством на всю страну, а то и мир. В этих ресторанах повара совершенствуют меню, менеджеры – обслуживание, а гурманы требуют, чтобы их прихоти были уникальны и удовлетворялись по высшему разряду.\n\nМы подобрали 

Теперь попробуем собрать информацию о всех наших десяти видео!

In [15]:
video_ids = []

for video in response_search['items']:
    video_ids.append(video['id']['videoId'])
    
print(video_ids)

['DuNsxZdRqs4', 'DskGDinJbOE', 'tG8ptZl9Yh0', 'YGNuVEYUeDk', 'L7-KwGjGZYQ', 'HqODWcABzj4', 'bjnTa4lgpqc', 'Vom6uCKv1KI', 'O6vj9T7KDSE', 'VCsK55Y3vGA']


In [16]:
video_10_request = youtube.videos().list(
    part = ['snippet', 'contentDetails', 'id', 'statistics'],
    id = video_ids
)

video_10_response = video_10_request.execute()

In [18]:
video_10_response

{'kind': 'youtube#videoListResponse',
 'etag': 'jactARTDV_ZZuFiLmYRq2ZKi-ks',
 'items': [{'kind': 'youtube#video',
   'etag': 'a4zAYlaBv_bFR080Uw7HKH8y4SI',
   'id': 'DuNsxZdRqs4',
   'snippet': {'publishedAt': '2023-03-30T10:08:31Z',
    'channelId': 'UCVq_U7ZYmaTNK5VPu0_VyBw',
    'title': 'Рестораны Москвы | Трейлер | PREMIER',
    'description': 'Вместе с самыми известными рестораторами и звёздами гида «Michelin» Сергей Минаев отправляется в большое путешествие по ресторанной индустрии Москвы, чтобы разобраться, как устроены самые модные заведения столицы, как создаются самые изысканные блюда и какие риски ждут тех, кто хочет связать свою жизнь с ресторанным бизнесом.\n\nСмотрите только в онлайн-кинотеатре PREMIER:\nhttps://premier.one/show/restorany-moskvy?utm_medium=smm&utm_source=youtube&utm_campaign=restorany-moskvy&utm_content=trailer&utm_term=description\n\nПодписывайтесь на PREMIER в других соцсетях:\nhttps://taplink.cc/premier\n\nПриложение PREMIER для Google Play: https://

## Базовая обработка выгрузки из API

Чтобы привести это все в относительно приличный вид, сделаем таблицу. Поможет нам в этом модуль pandas, нам понадобится только один метод — `json_normalize()`.

In [19]:
import pandas as pd
table = pd.json_normalize(video_10_response['items'])
table

kind                         etag           id  \
0  youtube#video  a4zAYlaBv_bFR080Uw7HKH8y4SI  DuNsxZdRqs4   
1  youtube#video  jaUjKr5x0jKB0Sv03VDL2uBpzFY  DskGDinJbOE   
2  youtube#video  KDdu4VGdDFYwcVhp6L3z6kXFhgs  tG8ptZl9Yh0   
3  youtube#video  ecuMgtS0rwHoZ7NqRDqYDa8pxSA  YGNuVEYUeDk   
4  youtube#video  PRV5SNikZSeHDpORLGdNLUg6ayc  L7-KwGjGZYQ   
5  youtube#video  q40upP-aomMQAL6nct6gs5SAg9M  HqODWcABzj4   
6  youtube#video  DW8scuDjrs8Y5gKJ10NRhA6Q8wI  bjnTa4lgpqc   
7  youtube#video  kCk9Bp8-bmWZf5Ze41CNDOJrHAA  Vom6uCKv1KI   
8  youtube#video  rUcJnytU8WY_U9Qw0U3-leH35P0  O6vj9T7KDSE   
9  youtube#video  tBSr52BnqS2z1Q1c3NWx4dfW8V0  VCsK55Y3vGA   

    snippet.publishedAt         snippet.channelId  \
0  2023-03-30T10:08:31Z  UCVq_U7ZYmaTNK5VPu0_VyBw   
1  2021-02-27T11:58:57Z  UCQM0TC4icu74AJ3h7N4hmTg   
2  2023-02-28T10:00:03Z  UCxF6ylvxiQV5m-7Rawr3W7g   
3  2022-10-04T20:55:33Z  UCFBLfEciXJvFvwYlN29j0jA   
4  2022-12-24T10:00:03Z  UCK9HSMN5j3MS_I-O_gGTW7w   
5  2021-06-14T06:00:08Z  UC8vjBcB0LOLYPiAAuYwuy7w   
6  2021-10-04T15:21:52Z  UCPHqvAYYcO0sUyWBp8VE5uQ   
7  2021-03-26T14:04:49Z  UCQM0TC4icu74AJ3h7N4hmTg   
8  2020-10-20T11:00:05Z  UCxF6ylvxiQV5m-7Rawr3W7g   
9  2023-02-15T17:37:11Z  UCPvoket8Npuv2HTPhcFhuZg   

                                       snippet.title  \
0               Рестораны Москвы | Трейлер | PREMIER   
1  Топ 10 | Лучшие Рестораны Москвы | Обзор Ресто...   
2  БОЛЬШОЙ обзор еды Москвы / Гид по ЛЮБИМЫМ рест...   
3  в Москву на выходные: ТОП-6 кафе и ресторанов ...   
4  Один из ЛУЧШИХ ресторанов / Вкусные необычные ...   
5  Где поесть в Москве? Самые интересные и необыч...   
6  2 звезды Мишлен. Ужин в TWINS GARDEN в Москве....   
7  Самые Необычные Кафе и Рестораны Москвы | Кули...   
8  Самый ДЕШЕВЫЙ ресторан В ЦЕНТРЕ МОСКВЫ / Все б...   
9     Старейшие магазины, рестораны и туалеты Москвы   

                                 snippet.description  \
0  Вместе с самыми известными рестораторами и звё...   
1  Ресторан – это не только заведение, где можно ...   
2  Подписаться на канал с обзорами ресторанов: ht...   
3  Где поесть в Москве, если вы скучаете по путеш...   
4  Успейте купить комплект Яндекс Станция 2 + Ста...   
5  Давно думали, куда сходить в Москве? Я собрала...   
6  Телеграм-канал Кто такая Элис — https://t.me/k...   
7  Необычные рестораны Москвы.\n\nКогда процесс п...   
8  Самые острые вопросы: https://www.youtube.com/...   
9  Заказывайте вкусную еду (и не только) в прилож...   

                   snippet.thumbnails.default.url  \
0  https://i.ytimg.com/vi/DuNsxZdRqs4/default.jpg   
1  https://i.ytimg.com/vi/DskGDinJbOE/default.jpg   
2  https://i.ytimg.com/vi/tG8ptZl9Yh0/default.jpg   
3  https://i.ytimg.com/vi/YGNuVEYUeDk/default.jpg   
4  https://i.ytimg.com/vi/L7-KwGjGZYQ/default.jpg   
5  https://i.ytimg.com/vi/HqODWcABzj4/default.jpg   
6  https://i.ytimg.com/vi/bjnTa4lgpqc/default.jpg   
7  https://i.ytimg.com/vi/Vom6uCKv1KI/default.jpg   
8  https://i.ytimg.com/vi/O6vj9T7KDSE/default.jpg   
9  https://i.ytimg.com/vi/VCsK55Y3vGA/default.jpg   

   snippet.thumbnails.default.width  snippet.thumbnails.default.height  ...  \
0                               120                                 90  ...   
1                               120                                 90  ...   
2                               120                                 90  ...   
3                               120                                 90  ...   
4                               120                                 90  ...   
5                               120                                 90  ...   
6                               120                                 90  ...   
7                               120                                 90  ...   
8                               120                                 90  ...   
9                               120                                 90  ...   

  contentDetails.definition  contentDetails.caption  \
0 

In [20]:
table.sort_values(by = 'statistics.viewCount', ascending = False)

kind                         etag           id  \
8  youtube#video  rUcJnytU8WY_U9Qw0U3-leH35P0  O6vj9T7KDSE   
2  youtube#video  KDdu4VGdDFYwcVhp6L3z6kXFhgs  tG8ptZl9Yh0   
6  youtube#video  DW8scuDjrs8Y5gKJ10NRhA6Q8wI  bjnTa4lgpqc   
4  youtube#video  PRV5SNikZSeHDpORLGdNLUg6ayc  L7-KwGjGZYQ   
9  youtube#video  tBSr52BnqS2z1Q1c3NWx4dfW8V0  VCsK55Y3vGA   
5  youtube#video  q40upP-aomMQAL6nct6gs5SAg9M  HqODWcABzj4   
7  youtube#video  kCk9Bp8-bmWZf5Ze41CNDOJrHAA  Vom6uCKv1KI   
3  youtube#video  ecuMgtS0rwHoZ7NqRDqYDa8pxSA  YGNuVEYUeDk   
0  youtube#video  a4zAYlaBv_bFR080Uw7HKH8y4SI  DuNsxZdRqs4   
1  youtube#video  jaUjKr5x0jKB0Sv03VDL2uBpzFY  DskGDinJbOE   

    snippet.publishedAt         snippet.channelId  \
8  2020-10-20T11:00:05Z  UCxF6ylvxiQV5m-7Rawr3W7g   
2  2023-02-28T10:00:03Z  UCxF6ylvxiQV5m-7Rawr3W7g   
6  2021-10-04T15:21:52Z  UCPHqvAYYcO0sUyWBp8VE5uQ   
4  2022-12-24T10:00:03Z  UCK9HSMN5j3MS_I-O_gGTW7w   
9  2023-02-15T17:37:11Z  UCPvoket8Npuv2HTPhcFhuZg   
5  2021-06-14T06:00:08Z  UC8vjBcB0LOLYPiAAuYwuy7w   
7  2021-03-26T14:04:49Z  UCQM0TC4icu74AJ3h7N4hmTg   
3  2022-10-04T20:55:33Z  UCFBLfEciXJvFvwYlN29j0jA   
0  2023-03-30T10:08:31Z  UCVq_U7ZYmaTNK5VPu0_VyBw   
1  2021-02-27T11:58:57Z  UCQM0TC4icu74AJ3h7N4hmTg   

                                       snippet.title  \
8  Самый ДЕШЕВЫЙ ресторан В ЦЕНТРЕ МОСКВЫ / Все б...   
2  БОЛЬШОЙ обзор еды Москвы / Гид по ЛЮБИМЫМ рест...   
6  2 звезды Мишлен. Ужин в TWINS GARDEN в Москве....   
4  Один из ЛУЧШИХ ресторанов / Вкусные необычные ...   
9     Старейшие магазины, рестораны и туалеты Москвы   
5  Где поесть в Москве? Самые интересные и необыч...   
7  Самые Необычные Кафе и Рестораны Москвы | Кули...   
3  в Москву на выходные: ТОП-6 кафе и ресторанов ...   
0               Рестораны Москвы | Трейлер | PREMIER   
1  Топ 10 | Лучшие Рестораны Москвы | Обзор Ресто...   

                                 snippet.description  \
8  Самые острые вопросы: https://www.youtube.com/...   
2  Подписаться на канал с обзорами ресторанов: ht...   
6  Телеграм-канал Кто такая Элис — https://t.me/k...   
4  Успейте купить комплект Яндекс Станция 2 + Ста...   
9  Заказывайте вкусную еду (и не только) в прилож...   
5  Давно думали, куда сходить в Москве? Я собрала...   
7  Необычные рестораны Москвы.\n\nКогда процесс п...   
3  Где поесть в Москве, если вы скучаете по путеш...   
0  Вместе с самыми известными рестораторами и звё...   
1  Ресторан – это не только заведение, где можно ...   

                   snippet.thumbnails.default.url  \
8  https://i.ytimg.com/vi/O6vj9T7KDSE/default.jpg   
2  https://i.ytimg.com/vi/tG8ptZl9Yh0/default.jpg   
6  https://i.ytimg.com/vi/bjnTa4lgpqc/default.jpg   
4  https://i.ytimg.com/vi/L7-KwGjGZYQ/default.jpg   
9  https://i.ytimg.com/vi/VCsK55Y3vGA/default.jpg   
5  https://i.ytimg.com/vi/HqODWcABzj4/default.jpg   
7  https://i.ytimg.com/vi/Vom6uCKv1KI/default.jpg   
3  https://i.ytimg.com/vi/YGNuVEYUeDk/default.jpg   
0  https://i.ytimg.com/vi/DuNsxZdRqs4/default.jpg   
1  https://i.ytimg.com/vi/DskGDinJbOE/default.jpg   

   snippet.thumbnails.default.width  snippet.thumbnails.default.height  ...  \
8                               120                                 90  ...   
2                               120                                 90  ...   
6                               120                                 90  ...   
4                               120                                 90  ...   
9                               120                                 90  ...   
5                               120                                 90  ...   
7                               120                                 90  ...   
3                               120                                 90  ...   
0                               120                                 90  ...   
1                               120                                 90  ...   

  contentDetails.definition  contentDetails.caption  \
8 